In [1]:
import pandas as pd
import re
import json
import uuid
from selenium import webdriver
from selenium.webdriver.chrome.options import Options
from selenium.webdriver.common.keys import Keys
from selenium.webdriver.common.by import By
from nltk.sentiment import SentimentIntensityAnalyzer
from selenium.webdriver.support.ui import WebDriverWait
from selenium.webdriver.support import expected_conditions as EC
from selenium.common.exceptions import ElementClickInterceptedException, StaleElementReferenceException, NoSuchElementException
from selenium.webdriver.chrome.service import Service as ChromeService
from webdriver_manager.chrome import ChromeDriverManager
from selenium.webdriver.common.action_chains import ActionChains
from selenium.webdriver.support.ui import Select
import time
import sqlite3
import spacy

In [15]:
driver = webdriver.Chrome()

In [16]:
url = "https://www.daraz.pk/smartphones/?page=1&spm=a2a0e.home.cate_7.1.35e34076cELmx5"
driver.get(url)

In [17]:
driver.maximize_window()

In [20]:
names = []
numofreviews = []
prices = []
brands = []
urls = []
scores = []
reviews = []
product_ids = []
wait_time = 10
for j in range(5):
    time.sleep(1)
    boxes = driver.find_elements(By.CLASS_NAME,"gridItem--Yd0sa")
    for i in range(len(boxes)):
        actions = ActionChains(driver)
        actions.move_to_element(boxes[i]).key_down(Keys.CONTROL).click(boxes[i]).key_up(Keys.CONTROL).perform()
        time.sleep(3)
        all_handles = driver.window_handles
        try:
            driver.switch_to.window(all_handles[1])
        except IndexError:
            continue
        time.sleep(3)
        try:
            tempname = driver.find_element(By.CLASS_NAME,"pdp-mod-product-badge-title").text
        except NoSuchElementException:
            tempname = "No Name"
        names.append(tempname)
        try:
            tempnumofreviews = driver.find_element(By.XPATH,"/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[4]/div[1]/div/div/a[1]").text
        except NoSuchElementException:
            tempnumofreviews = 0
        numofreviews.append(tempnumofreviews)
        try:
            temprices = driver.find_element(By.XPATH,"/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[7]/div/div/span").text
        except NoSuchElementException:
            temprices = 0
        prices.append(temprices)
        tempbrand = driver.find_element(By.XPATH,"/html/body/div[4]/div/div[3]/div[2]/div/div[1]/div[5]/div/a[1]").text
        if "No Brand" in tempbrand:
            x = tempname.split()
            tempbrand = x[0]
            brands.append(tempbrand)
        else:
            brands.append(tempbrand)
        tempurl = driver.current_url
        urls.append(tempurl)
        driver.execute_script("window.scrollBy(0, 600);")
        time.sleep(1)
        try:
            tempscore = driver.find_element(By.XPATH, "/html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[2]/div[1]/div[1]/div[1]/span[1]").text
        except NoSuchElementException:
            tempscore = 0
#             scores.append(tempscore)
        scores.append(tempscore)
        #reviews
        driver.execute_script("window.scrollBy(0, 800);")
        time.sleep(1)
        try:
            dropdown = driver.find_element(By.XPATH,'./html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[3]/div[4]/ul/li[10]/div/div')
        except NoSuchElementException:
            pass
        try:
            dropdown.click()
        except ElementClickInterceptedException:
            pass
        except StaleElementReferenceException:
            pass
        try:
            tempcli = driver.find_element(By.XPATH,"./html/body/div[4]/div/div[9]/div[1]/div[1]/div/div/div/div[3]/div[4]/ul/li[10]/div/div[2]/div/div/div/div/div/div[4]/div")
        except NoSuchElementException:
            pass
        try:
            tempcli.click()
        except ElementClickInterceptedException:
            pass
        except StaleElementReferenceException:
            pass
        time.sleep(1)
        try:
            x = driver.find_elements(By.CLASS_NAME,"review-content-sl")
            for i in range(len(x)):
                x[i]=x[i].text
            reviews.append(x)
        except:
            reviews.append("No Reviews")
        tempid = str(uuid.uuid4())[:4]
        product_ids.append(tempid)
        driver.close()
        driver.switch_to.window(all_handles[0])
        time.sleep(1)
    driver.execute_script("window.scrollTo(0, document.body.scrollHeight);")
    driver.execute_script("window.scrollBy(0, -2100);")
    time.sleep(1)
    nextbutton = driver.find_element(By.XPATH,"/html/body/div[3]/div/div[4]/div[1]/div/div[1]/div[3]/div[2]/div/ul/li[9]/a")
    nextbutton.click()

In [37]:
driver.close()
# driver.switch_to.window(all_handles[0])

In [28]:
len(product_ids)
# numofreviews
# prices
# brands
# urls
# scores
# reviews
# product_ids

196

In [29]:
#Creating dataframe
data = {
    'ProductID': product_ids,
    'Name': names,
    'Price': prices,
    'NumOfReviews': numofreviews,
    'Brand': brands,
    'URL': urls,
    'Score': scores,
    'Reviews': reviews
}
df = pd.DataFrame(data)

In [40]:
df['ReviewsString'] = df['Reviews'].apply(lambda x: '\n'.join(x))
df

,ProductID,Name,Price,NumOfReviews,Brand,URL,Score,Reviews,ReviewsString
0,99e7,Samsung A14 4GB/64GB - 5000 mAh Battery - 6.6'...,"Rs. 34,999",175 Ratings,Samsung,https://www.daraz.pk/products/samsung-a14-4gb6...,4.4,[100% Genuine / Brand New / Sealed Pack Box wi...,100% Genuine / Brand New / Sealed Pack Box wit...
1,d5eb,Infinix Note 30 Pro 8-256 GB PTA Approved With...,"Rs. 64,499",148 Ratings,Infinix,https://www.daraz.pk/products/infinix-note-30-...,4.7,"[I got what was shown, but I want to tell abou...","I got what was shown, but I want to tell about..."
2,e6e6,Realme Narzo 50A Prime (4GB-128GB) PTA Approve...,"Rs. 32,999",91 Ratings,Realme,https://www.daraz.pk/products/realme-narzo-50a...,4.2,[#11.11 sale main good mobile mil gaya hy mark...,#11.11 sale main good mobile mil gaya hy marke...
3,56fb,vivo Y17s - 6GB RAM - 128GB ROM - 5000mAh Batt...,"Rs. 44,999",62 Ratings,Vivo,https://www.daraz.pk/products/vivo-y17s-6gb-ra...,4.8,[original set and PTA approved set hai. boht h...,original set and PTA approved set hai. boht hi...
4,9873,Redmi A2+ - 3GB/64GB - Mediatek Helio G36 - An...,"Rs. 24,999",365 Ratings,Xiaomi,https://www.daraz.pk/products/redmi-a2-3gb64gb...,4.7,[I found this Redmi A2+ phone at an economical...,I found this Redmi A2+ phone at an economical ...
...,...,...,...,...,...,...,...,...,...
191,2ed3,"Redmi 12 , 8GB RAM 128GB ROM , 50 MP Main Came...","Rs. 42,999",4 Ratings,Redmi,https://www.daraz.pk/products/12-8gb-128gb-50-...,4.5,[Pin packed phone received in good condition. ...,Pin packed phone received in good condition. B...
192,d998,Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB F...,"Rs. 95,000",21 Ratings,Xiaomi,https://www.daraz.pk/products/12-8-gb-256-gb-1...,5.0,[waited for Redmi Note 12 Pro's release at Xia...,waited for Redmi Note 12 Pro's release at Xiao...
193,f6cf,Xiaomi Redmi 12 RAM 8 GB ROM 128 GB Front Came...,"Rs. 44,980",15 Ratings,Xiaomi,https://www.daraz.pk/products/12-8-gb-128-gb-8...,4.7,[Behtreen seller response I shocked itni fast ...,Behtreen seller response I shocked itni fast s...
194,ad84,Realme C51 RAM 4GB ROM 64GB Front Camera 5 MP ...,"Rs. 29,999",No Ratings,Realme,https://www.daraz.pk/products/c51-4gb-64gb-5-m...,0,[],


In [48]:
df.to_excel("data.xlsx", index=False)

In [51]:
#Fixing of dataframe

In [5]:
tempdf = pd.read_excel("data.xlsx")

In [35]:
def extract_price(price_string):
    return int(price_string.replace('Rs.', '').replace(',', ''))

# Apply the function to the 'price' column
tempdf['Price'] = tempdf['Price'].apply(extract_price)

In [43]:
def extract_rating(rating_string):
    if rating_string == 'No Ratings':
        return 0
    else:
        # Remove ' ratings', commas, and convert to integer
        return int(rating_string.replace(' Ratings', '').replace(',', ''))

# Apply the function to the 'rating' column
tempdf['NumOfReviews'] = tempdf['NumOfReviews'].apply(extract_rating)

In [45]:
tempdf.fillna(0)

,ProductID,Name,Price,NumOfReviews,Brand,URL,Score,Reviews,ReviewsString
0,99e7,Samsung A14 4GB/64GB - 5000 mAh Battery - 6.6'...,34999,175,Samsung,https://www.daraz.pk/products/samsung-a14-4gb6...,4.4,['100% Genuine / Brand New / Sealed Pack Box w...,100% Genuine / Brand New / Sealed Pack Box wit...
1,d5eb,Infinix Note 30 Pro 8-256 GB PTA Approved With...,64499,148,Infinix,https://www.daraz.pk/products/infinix-note-30-...,4.7,"['I got what was shown, but I want to tell abo...","I got what was shown, but I want to tell about..."
2,e6e6,Realme Narzo 50A Prime (4GB-128GB) PTA Approve...,32999,91,Realme,https://www.daraz.pk/products/realme-narzo-50a...,4.2,['#11.11 sale main good mobile mil gaya hy mar...,#11.11 sale main good mobile mil gaya hy marke...
3,56fb,vivo Y17s - 6GB RAM - 128GB ROM - 5000mAh Batt...,44999,62,Vivo,https://www.daraz.pk/products/vivo-y17s-6gb-ra...,4.8,['original set and PTA approved set hai. boht ...,original set and PTA approved set hai. boht hi...
4,9873,Redmi A2+ - 3GB/64GB - Mediatek Helio G36 - An...,24999,365,Xiaomi,https://www.daraz.pk/products/redmi-a2-3gb64gb...,4.7,['I found this Redmi A2+ phone at an economica...,I found this Redmi A2+ phone at an economical ...
...,...,...,...,...,...,...,...,...,...
191,2ed3,"Redmi 12 , 8GB RAM 128GB ROM , 50 MP Main Came...",42999,4,Redmi,https://www.daraz.pk/products/12-8gb-128gb-50-...,4.5,['Pin packed phone received in good condition....,Pin packed phone received in good condition. B...
192,d998,Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB F...,95000,21,Xiaomi,https://www.daraz.pk/products/12-8-gb-256-gb-1...,5.0,"[""waited for Redmi Note 12 Pro's release at Xi...",waited for Redmi Note 12 Pro's release at Xiao...
193,f6cf,Xiaomi Redmi 12 RAM 8 GB ROM 128 GB Front Came...,44980,15,Xiaomi,https://www.daraz.pk/products/12-8-gb-128-gb-8...,4.7,['Behtreen seller response I shocked itni fast...,Behtreen seller response I shocked itni fast s...
194,ad84,Realme C51 RAM 4GB ROM 64GB Front Camera 5 MP ...,29999,0,Realme,https://www.daraz.pk/products/c51-4gb-64gb-5-m...,0.0,[],0


In [97]:
tempdf.to_excel("data.xlsx", index=False)

In [96]:
#Bonus Creating database
columns_to_include = ['ProductID', 'Name', 'Price', 'NumOfReviews', 'Brand', 'URL', 'Score','ReviewsString','Specifications_String']
conn = sqlite3.connect('product_database.db')
tempdf[columns_to_include].to_sql('products', conn, index=False, if_exists='replace')

196

In [17]:
tempdf = pd.read_excel("data.xlsx")

In [18]:
tempdf

,ProductID,Name,Price,NumOfReviews,Brand,URL,Score,Reviews,ReviewsString
0,99e7,Samsung A14 4GB/64GB - 5000 mAh Battery - 6.6'...,34999,175,Samsung,https://www.daraz.pk/products/samsung-a14-4gb6...,4.4,['100% Genuine / Brand New / Sealed Pack Box w...,100% Genuine / Brand New / Sealed Pack Box wit...
1,d5eb,Infinix Note 30 Pro 8-256 GB PTA Approved With...,64499,148,Infinix,https://www.daraz.pk/products/infinix-note-30-...,4.7,"['I got what was shown, but I want to tell abo...","I got what was shown, but I want to tell about..."
2,e6e6,Realme Narzo 50A Prime (4GB-128GB) PTA Approve...,32999,91,Realme,https://www.daraz.pk/products/realme-narzo-50a...,4.2,['#11.11 sale main good mobile mil gaya hy mar...,#11.11 sale main good mobile mil gaya hy marke...
3,56fb,vivo Y17s - 6GB RAM - 128GB ROM - 5000mAh Batt...,44999,62,Vivo,https://www.daraz.pk/products/vivo-y17s-6gb-ra...,4.8,['original set and PTA approved set hai. boht ...,original set and PTA approved set hai. boht hi...
4,9873,Redmi A2+ - 3GB/64GB - Mediatek Helio G36 - An...,24999,365,Xiaomi,https://www.daraz.pk/products/redmi-a2-3gb64gb...,4.7,['I found this Redmi A2+ phone at an economica...,I found this Redmi A2+ phone at an economical ...
...,...,...,...,...,...,...,...,...,...
191,2ed3,"Redmi 12 , 8GB RAM 128GB ROM , 50 MP Main Came...",42999,4,Redmi,https://www.daraz.pk/products/12-8gb-128gb-50-...,4.5,['Pin packed phone received in good condition....,Pin packed phone received in good condition. B...
192,d998,Xiaomi Redmi Note 12 Pro RAM 8 GB ROM 256 GB F...,95000,21,Xiaomi,https://www.daraz.pk/products/12-8-gb-256-gb-1...,5.0,"[""waited for Redmi Note 12 Pro's release at Xi...",waited for Redmi Note 12 Pro's release at Xiao...
193,f6cf,Xiaomi Redmi 12 RAM 8 GB ROM 128 GB Front Came...,44980,15,Xiaomi,https://www.daraz.pk/products/12-8-gb-128-gb-8...,4.7,['Behtreen seller response I shocked itni fast...,Behtreen seller response I shocked itni fast s...
194,ad84,Realme C51 RAM 4GB ROM 64GB Front Camera 5 MP ...,29999,0,Realme,https://www.daraz.pk/products/c51-4gb-64gb-5-m...,0.0,[],NaN


In [91]:
import re

def extract_phone_specifications(phone_string):
    specifications = {}

    # Extract RAM and ROM when separated by a hyphen within parentheses
    ram_rom_parentheses_match = re.search(r'\((\d+)\s*GB\s*-\s*(\d+)\s*GB\)', phone_string, re.IGNORECASE)
    if ram_rom_parentheses_match:
        specifications['RAM'] = int(ram_rom_parentheses_match.group(1))
        specifications['ROM'] = int(ram_rom_parentheses_match.group(2))
    else:
        # Extract RAM and ROM when separated by a hyphen with spaces
        ram_rom_space_hyphen_match = re.search(r'(\d+)\s*-\s*(\d+)\s*GB', phone_string, re.IGNORECASE)
        if ram_rom_space_hyphen_match:
            specifications['RAM'] = int(ram_rom_space_hyphen_match.group(1))
            specifications['ROM'] = int(ram_rom_space_hyphen_match.group(2))
        else:
            # Extract RAM and ROM when separated by a hyphen
            ram_rom_hyphen_match = re.search(r'(\d+)\s*-\s*(\d+)\s*GB', phone_string, re.IGNORECASE)
            if ram_rom_hyphen_match:
                specifications['RAM'] = int(ram_rom_hyphen_match.group(1))
                specifications['ROM'] = int(ram_rom_hyphen_match.group(2))
            else:
                # Extract RAM and ROM when separated by a slash
                ram_rom_slash_match = re.search(r'(\d+)GB/(\d+)GB', phone_string, re.IGNORECASE)
                if ram_rom_slash_match:
                    specifications['RAM'] = int(ram_rom_slash_match.group(1))
                    specifications['ROM'] = int(ram_rom_slash_match.group(2))
                else:
                    # Extract RAM and ROM when specified without a hyphen (e.g., "4GB+128GB")
                    ram_rom_combined_match = re.search(r'(\d+)\s*GB\s*\+\s*(\d+)\s*GB', phone_string, re.IGNORECASE)
                    if ram_rom_combined_match:
                        specifications['RAM'] = int(ram_rom_combined_match.group(1))
                        specifications['ROM'] = int(ram_rom_combined_match.group(2))
                    else:
                        # Extract RAM
                        ram_match = re.search(r'(\d+)GB\s*RAM', phone_string, re.IGNORECASE)
                        if ram_match:
                            specifications['RAM'] = int(ram_match.group(1))

                        # Extract ROM
                        rom_match = re.search(r'(\d+)GB\s*ROM', phone_string, re.IGNORECASE)
                        if rom_match:
                            specifications['ROM'] = int(rom_match.group(1))

    # Extract Battery
    battery_match = re.search(r'(\d+)\s*mAh\s*Battery', phone_string, re.IGNORECASE)
    if battery_match:
        specifications['Battery'] = int(battery_match.group(1))

    # Extract Display
    display_match = re.search(r'(\d+(\.\d+)?)\'\s*Display', phone_string, re.IGNORECASE)
    if display_match:
        specifications['Display'] = float(display_match.group(1))

    # Extract Android version
    android_match = re.search(r'Android\s+(\d+(\.\d+)?)', phone_string, re.IGNORECASE)
    if android_match:
        specifications['Android'] = float(android_match.group(1))

    # Extract Camera Megapixels
    camera_match = re.search(r'(\d+)\s*Megapixels\s*Camera', phone_string, re.IGNORECASE)
    if camera_match:
        specifications['Camera'] = int(camera_match.group(1))

    return specifications
# Example usage:
tempdf["Specifications"] = tempdf["Name"].apply(extract_phone_specifications)
# for i in tempdf["Name"]:
#     phone_string = i
#     specs = extract_phone_specifications(phone_string)
#     print("Name :",i)
#     print("Specs : ",specs)
tempdf["Specifications_String"] = tempdf["Specifications"].apply(json.dumps)

In [ ]:
#Chatbot Development

In [2]:
nlp = spacy.load("en_core_web_sm")

In [3]:
conn = sqlite3.connect('product_database.db')
cursor = conn.cursor()

In [4]:
def findavg(column):
    query = f"SELECT AVG({column}) FROM {'products'};"
    cursor.execute(query)
    avg = cursor.fetchone()[0]
    return avg

In [5]:
query = "SELECT DISTINCT Brand FROM products"
cursor.execute(query)
brands = cursor.fetchall()

# brands = list(brands)
for i in range(len(brands)):
    brands[i] = brands[i][0].lower()
brands.remove("original")
brands.remove("imported")
brands.remove("combo")
brands.remove("a57")

In [6]:
def extract_integer_from_text(doc):
    for i in doc:
        if i.is_digit:
            return int(i.text)
    return None
def extract_integers_from_text(doc):
    numbers = []
    current_number = ''
    
    for token in doc:
        if token.is_digit:
            current_number += token.text
        elif current_number:
            numbers.append(int(current_number))
            current_number = ''

    if current_number:
        numbers.append(int(current_number))

    return numbers if numbers else None
def extract_numbers_from_text(doc):
    numbers = []
    current_number = ''

    for token in doc:
        if token.like_num or (token.is_punct and token.text == '.'):
            current_number += token.text
        elif current_number:
            numbers.append(float(current_number) if '.' in current_number else int(current_number))
            current_number = ''

    if current_number:
        numbers.append(float(current_number) if '.' in current_number else int(current_number))

    return numbers if numbers else None
def undervalue(value):
    query = f"SELECT Name, Price, URL, Score FROM products WHERE Price < {value} ORDER BY Score DESC, NumOfReviews DESC LIMIT 1"
    cursor.execute(query)
    returning = cursor.fetchone()
    if not returning:
        return f"No product available under {value}"
    final=""
    final+="Name : "
    final+= returning[0]
    final+="\n"
    final+="Price : "
    final+= str(returning[1])
    final+="\n"
    final+="Rating : "
    final+=str(returning[3])
    final+="\n"
    final+="URL : "
    final+=returning[2]
    final+="\n"
    return final
def overvalue(value):
    query = f"SELECT Name, Price, URL, Score FROM products WHERE Price > {value} ORDER BY Score DESC, NumOfReviews DESC LIMIT 1"
    cursor.execute(query)
    returning = cursor.fetchone()
    if not returning:
        return f"No product available under {value}"
    final=""
    final+="Name : "
    final+= returning[0]
    final+="\n"
    final+="Price : "
    final+= str(returning[1])
    final+="\n"
    final+="Rating : "
    final+=str(returning[3])
    final+="\n"
    final+="URL : "
    final+=returning[2]
    final+="\n"
    return final
def undervalues(value):
    query = f"SELECT Name, Price, URL,Score FROM products WHERE Price < {value}"
    cursor.execute(query)
    returning = cursor.fetchall()
    if not returning:
        return f"No products available under {value}"
    final=""
    for i in range(len(returning)):
        final+="Product "
        final+=str(i+1)
        final+=" : "
        final+="\n"
        final+="Name : "
        final+= returning[i][0]
        final+="\n"
        final+="Price : "
        final+= str(returning[i][1])
        final+="\n"
        final+="Rating : "
        final+=str(returning[i][3])
        final+="\n"
        final+="URL : "
        final+=returning[i][2]
        final+="\n"
    return final
def overvalues(value):
    query = f"SELECT Name, Price, URL,Score FROM products WHERE Price > {value}"
    cursor.execute(query)
    returning = cursor.fetchall()
    if not returning:
        return f"No products available under {value}"
    final=""
    for i in range(len(returning)):
        final+="Product "
        final+=str(i+1)
        final+=" : "
        final+="\n"
        final+="Name : "
        final+= str(returning[i][0])
        final+="\n"
        final+="Price : "
        final+= str(returning[i][1])
        final+="\n"
        final+="Rating : "
        final+=str(returning[i][3])
        final+="\n"
        final+="URL : "
        final+=str(returning[i][2])
        final+="\n"
    return final
def between(val1,val2):
    query = f"SELECT Name, Price, URL, Score FROM products WHERE Price BETWEEN {val1} AND {val2}"
    cursor.execute(query)
    returning = cursor.fetchall()
    if not returning:
        return f"No products available between {val1} and {val2}"
    final=""
    for i in range(len(returning)):
        final+="Product "
        final+=str(i+1)
        final+=" : "
        final+="\n"
        final+="Name : "
        final+= str(returning[i][0])
        final+="\n"
        final+="Price : "
        final+= str(returning[i][1])
        final+="\n"
        final+="Rating : "
        final+=str(returning[i][3])
        final+="\n"
        final+="URL : "
        final+=str(returning[i][2])
        final+="\n"
    return final
def undervaluesbrand(brand,value):
    brand = brand[0].upper() + brand[1:]
    query = f"SELECT Name, Price, URL, Score FROM products WHERE Price < {value} AND Brand = ?"
    cursor.execute(query, (brand,))
    returning = cursor.fetchall()
    if not returning:
        return f"No products available under {value} for brand {brand}"
    final=f"Brand : {brand}"
    final+="\n"
    for i in range(len(returning)):
        final+="Product "
        final+=str(i+1)
        final+=" : "
        final+="\n"
        final+="Name : "
        final+= returning[i][0]
        final+="\n"
        final+="Price : "
        final+= str(returning[i][1])
        final+="\n"
        final+="Rating : "
        final+=str(returning[i][3])
        final+="\n"
        final+="URL : "
        final+=returning[i][2]
        final+="\n"
    return final
def overvaluesbrand(brand,value):
    brand = brand[0].upper() + brand[1:]
    query = f"SELECT Name, Price, URL, Score FROM products WHERE Price > {value} AND Brand = ?"
    cursor.execute(query, (brand,))
    returning = cursor.fetchall()
    if not returning:
        return f"No products available under {value} for brand {brand}"
    final=f"Brand : {brand}"
    final+="\n"
    for i in range(len(returning)):
        final+="Product "
        final+=str(i+1)
        final+=" : "
        final+="\n"
        final+="Name : "
        final+= returning[i][0]
        final+="\n"
        final+="Price : "
        final+= str(returning[i][1])
        final+="\n"
        final+="Rating : "
        final+=str(returning[i][3])
        final+="\n"
        final+="URL : "
        final+=returning[i][2]
        final+="\n"
    return final

def betweenbrand(brand,val1,val2):
    brand = brand[0].upper() + brand[1:]
    query = f"SELECT Name, Price, URL, Score FROM products WHERE Price BETWEEN {val1} AND {val2} AND Brand = ?"
    cursor.execute(query, (brand,))
    returning = cursor.fetchall()
    if not returning:
        return f"No products available between {val1} and {val2} of Brand {brand}"
    final=""
    final=f"Brand : {brand}"
    final+="\n"
    for i in range(len(returning)):
        final+="Product "
        final+=str(i+1)
        final+=" : "
        final+="\n"
        final+="Name : "
        final+= str(returning[i][0])
        final+="\n"
        final+="Price : "
        final+= str(returning[i][1])
        final+="\n"
        final+="Rating : "
        final+=str(returning[i][3])
        final+="\n"
        final+="URL : "
        final+=str(returning[i][2])
        final+="\n"
    return final
def undervaluesbrandtop(brand,value,num):
    brand = brand[0].upper() + brand[1:]
    query = f"SELECT Name, Price, URL, Score FROM products WHERE Price < {value} AND Brand = ? ORDER BY Score DESC LIMIT {num}"
    cursor.execute(query, (brand,))
    returning = cursor.fetchall()
    if not returning:
        return f"No {num} products available under {value} for brand {brand}"
    final=f"Brand : {brand}"
    final+="\n"
    for i in range(len(returning)):
        final+="Product "
        final+=str(i+1)
        final+=" : "
        final+="\n"
        final+="Name : "
        final+= returning[i][0]
        final+="\n"
        final+="Price : "
        final+= str(returning[i][1])
        final+="\n"
        final+="Rating : "
        final+=str(returning[i][3])
        final+="\n"
        final+="URL : "
        final+=returning[i][2]
        final+="\n"
    return final
def overvaluesbrandtop(brand,value,num):
    brand = brand[0].upper() + brand[1:]
    query = f"SELECT Name, Price, URL, Score FROM products WHERE Price > {value} AND Brand = ? ORDER BY Score DESC LIMIT {num}"
    cursor.execute(query, (brand,))
    returning = cursor.fetchall()
    if not returning:
        return f"No {num} products available over {value} for brand {brand}"
    final=f"Brand : {brand}"
    final+="\n"
    for i in range(len(returning)):
        final+="Product "
        final+=str(i+1)
        final+=" : "
        final+="\n"
        final+="Name : "
        final+= returning[i][0]
        final+="\n"
        final+="Price : "
        final+= str(returning[i][1])
        final+="\n"
        final+="Rating : "
        final+=str(returning[i][3])
        final+="\n"
        final+="URL : "
        final+=returning[i][2]
        final+="\n"
    return final
def betweenvaluesbrandtop(brand,val1,num,val2):
    brand = brand[0].upper() + brand[1:]
    query = f"SELECT Name, Price, URL, Score FROM products WHERE Price BETWEEN {val1} AND {val2} AND Brand = ? ORDER BY Score DESC LIMIT {num}"
    cursor.execute(query, (brand,))
    returning = cursor.fetchall()
    if not returning:
        return f"No {num} products available between {val1} and {val2} for brand {brand}"
    final=f"Brand : {brand}"
    final+="\n"
    for i in range(len(returning)):
        final+="Product "
        final+=str(i+1)
        final+=" : "
        final+="\n"
        final+="Name : "
        final+= returning[i][0]
        final+="\n"
        final+="Price : "
        final+= str(returning[i][1])
        final+="\n"
        final+="Rating : "
        final+=str(returning[i][3])
        final+="\n"
        final+="URL : "
        final+=returning[i][2]
        final+="\n"
    return final


In [7]:
greet = ["hi","hello","hey","hola"]
urdugreet = ["kesa","kese","kesi","kes"]
def findresponse(query):
    if "fardeen" in query.text:
        return "OMG DID I JUST HEAR FARDEEN! I LOST YAAR ITNA CUTU BACHA HAI KASH MEI FARDEEN HOTA!"
    if any(word in query.text for word in greet):
        return "Hello!"
    if any(word in query.text for word in urdugreet):
        return "theek thaak Alhamdullilah"
    if "rating" in query.text:
        y = query.text.split()
        x = extract_numbers_from_text(query)
        return x
    if "top" in query.text:
        y = query.text.split()
        x = extract_integers_from_text(query)
        if "under" in query.text:
            tempbrand = []
            for i in y:
                if i in brands:
                    tempbrand.append(i)
            returningstring = ""
            if len(tempbrand)>1:
                for i in tempbrand:
                    returningstring+=undervaluesbrandtop(i,x[1],x[0])
                    returningstring+="\n"
            else:
                returningstring = undervaluesbrandtop(tempbrand[0],x[1],x[0])
            return returningstring
        if "between" in query.text:
            tempbrand = []
            for i in y:
                if i in brands:
                    tempbrand.append(i)
            returningstring = ""
            if len(tempbrand)>1:
                for i in tempbrand:
                    returningstring+=betweenvaluesbrandtop(i,x[1],x[0],x[2])
                    returningstring+="\n"
            else:
                returningstring = betweenvaluesbrandtop(tempbrand[0],x[1],x[0],x[2])
            return returningstring
        if "over" in query.text:
            tempbrand = []
            for i in y:
                if i in brands:
                    tempbrand.append(i)
            returningstring = ""
            if len(tempbrand)>1:
                for i in tempbrand:
                    returningstring+=overvaluesbrandtop(i,x[1],x[0])
                    returningstring+="\n"
            else:
                returningstring = overvaluesbrandtop(tempbrand[0],x[1],x[0])
            return returningstring
    if any(word in query.text for word in brands) and "between" in query.text:
        y = query.text.split()
        x = extract_integers_from_text(query)
        if x == None:
            return "I didn't quite get what you meant, can you please repeat yourself"
        tempbrand = []
        for i in y:
            if i in brands:
                tempbrand.append(i)
        returningstring = ""
        if len(tempbrand)>1:
            for i in tempbrand:
                returningstring+=betweenbrand(i,x[0],x[1])
                returningstring+="\n"
        else:
            returningstring = betweenbrand(tempbrand[0],x[0],x[1])
        return returningstring
    if any(word in query.text for word in brands) and "under" in query.text:
        y = query.text.split()
        x = extract_integer_from_text(query)
        if x == None:
            return "I didn't quite get what you meant, can you please repeat yourself"
        tempbrand = []
        for i in y:
            if i in brands:
                tempbrand.append(i)
        returningstring = ""
        if len(tempbrand)>1:
            for i in tempbrand:
                returningstring+=undervaluesbrand(i,x)
                returningstring+="\n"
        else:
            returningstring = undervaluesbrand(tempbrand[0],x)
        return returningstring
    if any(word in query.text for word in brands) and "over" in query.text:
        y = query.text.split()
        x = extract_integer_from_text(query)
        if x == None:
            return "I didn't quite get what you meant, can you please repeat yourself"
        tempbrand = []
        for i in y:
            if i in brands:
                tempbrand.append(i)
        returningstring = ""
        if len(tempbrand)>1:
            for i in tempbrand:
                returningstring+=overvaluesbrand(i,x)
                returningstring+="\n"
        else:
            returningstring = overvaluesbrand(tempbrand[0],x)
        return returningstring
    if "under" in query.text and "phones" not in query.text:
        x = extract_integer_from_text(query)
        if x == None:
            return "I didn't quite get what you meant, can you please repeat yourself"
        return undervalue(x)
    if "under" in query.text:
        x = extract_integer_from_text(query)
        if x == None:
            return "I didn't quite get what you meant, can you please repeat yourself"
        return undervalues(x)
    if "over" in query.text and "phones" not in query.text:
        x = extract_integer_from_text(query)
        if x == None:
            return "I didn't quite get what you meant, can you please repeat yourself"
        return overvalue(x)
    if "over" in query.text:
        x = extract_integer_from_text(query)
        if x == None:
            return "I didn't quite get what you meant, can you please repeat yourself"
        return overvalues(x)
    if "between" in query.text:
        x = extract_integers_from_text(query)
        if x == None:
            return "I didn't quite get what you meant, can you please repeat yourself"
        return between(x[0],x[1])
    return "I didn't quite get what you meant, can you please repeat yourself"

In [8]:
def simple_chatbot(query):
    doc = nlp(query.lower())  # Convert the query to lowercase
    return findresponse(doc)

In [10]:
user_query = "best samsung phones between 20000 and 30000"
response = simple_chatbot(user_query)
print(response)

Brand : Samsung
Product 1 : 
Name : Samsung Galaxy A04 4GB - 64GB OFFICIAL PTA APPROVED OFFICIAL WARRANTY
Price : 29999
Rating : 4.5
URL : https://www.daraz.pk/products/samsung-galaxy-a04-4gb-64gb-official-pta-approved-official-warranty-i438355041-s2122292351.html?spm=a2a0e.searchlistcategory.sku.33.4711676dx2TR9k&search=1
Product 2 : 
Name : Samsung Galaxy A04 4GB - 64GB OFFICIAL PTA APPROVED OFFICIAL WARRANTY
Price : 29999
Rating : 4.5
URL : https://www.daraz.pk/products/samsung-galaxy-a04-4gb-64gb-official-pta-approved-official-warranty-i438355041-s2122292351.html?spm=a2a0e.searchlistcategory.sku.33.467c676dmnJHkV&search=1
Product 3 : 
Name : Samsung Galaxy A04 - 6.5" Inch Display - 3GB RAM - 32GB ROM
Price : 23499
Rating : 5.0
URL : https://www.daraz.pk/products/samsung-galaxy-a04-65-inch-display-3gb-ram-32gb-rom-i396504249-s1921861685.html?spm=a2a0e.searchlistcategory.sku.3.3226676dZ0jq8r&search=1



In [14]:
#review based searching
#flask

In [ ]:
#flask